## Download files

In [3]:
# Read json source
import json
from os import path

SourceData = json.load(open(path.abspath('./data/data.json')))

In [4]:
import sys
!{sys.executable} -m pip install pymupdf

In [5]:
import os

OUTPUT_PATH = path.abspath('./data/generated/input')

def saveText(type, candidat, filename, contentStr):
    final_dir = path.join(OUTPUT_PATH, type, candidat)
    if not os.path.exists(final_dir):
        os.makedirs(final_dir)
    with open(path.join(final_dir, filename), encoding='utf-8', mode='w') as f:
        f.write(contentStr)

In [6]:
import requests
import fitz

def pdfToText(byteArray):
    doc = fitz.open("x.pdf", byteArray)
    pages = [ doc[ i ] for i in range( doc.pageCount ) ]

    return '\n---PAGE---\n'.join([page.get_text('text') for page in pages])

def downloadPdf(candidat, current_url):
    resp = requests.get(current_url)
    if resp.status_code != 200:
        print("ERROR ({}) for {}".format(resp.status_code, current_url))
    else:
        filename = current_url.split("/")[-1]
        saveText('pdf', candidat, filename+".txt", pdfToText(resp.content))
            


In [21]:
import re
from bs4 import BeautifulSoup

def downloadHtml(candidat, current_url):
    resp = requests.get(current_url)
    if resp.status_code != 200:
        print("ERROR ({}) for {}".format(resp.status_code, current_url))
    else:
        filename = (current_url[:-1] if (current_url.endswith("/")) else current_url).split("/")[-1]
        fullContent = resp.content
        soup = BeautifulSoup(fullContent, features="html.parser")
        for script in soup(["script", "style", "nav"]):
            script.extract()
        
        if candidat == "macron" or candidat == "pecresse":
            # On doit supprimer ler html si il y a : data-elementor-type="footer"
            for elem in soup.find_all('div', {'data-elementor-type': "footer"}):
                elem.decompose()
            for elem in soup.find_all('div', {'data-elementor-type': "header"}):
                elem.decompose()
        
        if candidat == "macron":
            for elem in soup.find_all('section', {'data-id': "5d859698"}):
                elem.decompose()

        if candidat == "poutou":
            # On doit supprimer ler html si il y a : data-elementor-type="footer"
            for elem in soup.find_all('div', {'id': "header-outer-wrapper-sticky-wrapper"}):
                elem.decompose()
            for elem in soup.find_all('footer'):
                elem.decompose()
        
        cleanText = soup.get_text()
        cleanTextWithoutEmpty = re.sub(r'(\n\s*)+\n+', '\n\n', cleanText)
        saveText('html', candidat, filename+".html.txt", cleanTextWithoutEmpty)

In [23]:
# Download files
for candidat in SourceData:
    for current_url in SourceData[candidat]:
        if current_url.endswith(".pdf"):
            downloadPdf(candidat, current_url)
        else:
            downloadHtml(candidat, current_url)